# Time Constructions

Towards a usage-based, constructional taxonomy of time indicators in Biblical Hebrew.

In [ ]:
from tf.fabric import Fabric
from tf.app import use

custom_data = ['../data/']

TF = Fabric(locations=custom_data)
api = TF.load('''

function st vs vt gloss

''')

